#Подготовка среды

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vk_api

In [ ]:
import json
import vk_api
import time
import numpy as np
import pandas as pd
from collections import Counter
import statistics as st

In [ ]:
def captcha_handler(captcha):

    key = input("Enter captcha code {0}: ".format(captcha.get_url())).strip()

    # Повтор запроса с капчей
    return captcha.try_again(key)

#чтение файла с логином и паролем
f = open("login.txt", "r")

#открытие сесси вк
vk_session = vk_api.VkApi(f.readline(), f.readline(), captcha_handler=captcha_handler)
vk_session.auth(token_only=True)
vk = vk_session.get_api()

# Обработка данных

In [ ]:
firstCom = 'name_first' # название первого паблика
secondCom = 'name_second' # название второго паблика
count = '1000' # объем выборки для каждого паблика

In [ ]:
#сбор id подписчиков и их друзей
def collectingData(firstCom, secondCom, count, vk):

    #сбор данных об участниках групп
    getFirstCom = vk.groups.getMembers(group_id = firstCom, count = count, offset = '2000')
    getSecondCom = vk.groups.getMembers(group_id = secondCom, count = count, offset = '100')

    #поиск повторящихся пользователей пабликов
    getCommon, getFirstCom, getSecondCom = searchCommon(getFirstCom["items"], getSecondCom["items"])
        
    dataFriends = {
        firstCom : {},
        secondCom : {}, 
        "commonCom" : {}
    }

    countFirst = 0
    countSecond = 0
    countCommon = 0
   
    time.sleep(3)

    # данные о друзьях 1 паблика
    for i in range(len(getFirstCom)):           
        try:
            getFriends = vk.friends.get(user_id = str(getFirstCom[i]))
            g = {str(getFirstCom[i]): getFriends['items']}
            dataFriends[firstCom].update(g)
        except:
            countFirst += 1
    
    time.sleep(3)

    # данные о друзьях 2 паблика
    for i in range(len(getSecondCom)):           
        try:
            getFriends = vk.friends.get(user_id = str(getSecondCom[i]))
            g = {str(getSecondCom[i]): getFriends['items']}
            dataFriends[secondCom].update(g)
        except:
            countSecond += 1
   
    time.sleep(3)

    # данные о друзьях общего паблика
    for i in range(len(getCommon)):           
        try:
            getFriends = vk.friends.get(user_id = str(getCommon[i]))
            g = {str(getCommon[i]): getFriends['items']}
            dataFriends["commonCom"].update(g)
        except:
            countCommon += 1

#сохранение файла на диске
    with open('drive/MyDrive/output/data_file.json', 'w') as outfile:
        json.dump(dataFriends, outfile)
    
    return countFirst, countSecond, countCommon

In [ ]:
#поиск общих подписчиков между первым и вторым пабликом
def searchCommon(a, b):
  res1 = []
  res2 = []
  common = []
  for i in range(len(a)):
      try: 
          b.index(a[i])
          common.append(a[i])
      except:
          res1.append(a[i])  

  for i in range(len(b)):
      try: 
          a.index(b[i])
      except:
          res2.append(b[i])  
   
  return common, res1, res2

In [ ]:
#запуск сбора данных
countFirst, countSecond, countCommon = collectingData(firstCom, secondCom, count, vk)
print(countFirst, countSecond, countCommon)

In [ ]:
# Матрица смежности
def processingData(firstCom, secondCom):

    #чтение файла json
    with open("drive/MyDrive/output/data_file.json", "r") as infile: 
        data = infile.read()

    dataFriends = json.loads(data)

    #перенос общих агентов
    dl = []
    for i in dataFriends[firstCom]:
        if vk.groups.isMember(group_id = secondCom, user_id  = i ):
            dataFriends["commonCom"][i] = dataFriends[firstCom][i]
            dl.append(i)
    for i in dl:
        dataFriends[firstCom].pop(i)
    dl = []
    for i in dataFriends[secondCom]:
        if vk.groups.isMember(group_id = firstCom, user_id  = i ):
            dataFriends["commonCom"][i] = dataFriends[secondCom][i]
            dl.append(i)
    for i in dl:
        dataFriends[secondCom].pop(i)

    #определение длины каждого словаря
    lenFirst = len(dataFriends[firstCom])
    lenSecond = len(dataFriends[secondCom])
    lenCommon = len(dataFriends["commonCom"])
    lenTotal = lenFirst + lenSecond + lenCommon + 2
    rv = {}#rv - вектор смежности

    #ids
    idsFirst = [*dataFriends[firstCom]]
    idsSecond = [*dataFriends[secondCom]]
    idsCommon = [*dataFriends["commonCom"]]
    idsTotal = set([int(i) for i in idsFirst+idsSecond+idsCommon])

    #соединение с сообществом
    rv[firstCom] = [int(i) for i in idsFirst+idsCommon]
    rv[secondCom] = [int(i) for i in idsSecond+idsCommon]
    
    #соединение друзей
    ttl = dataFriends[firstCom]|dataFriends[secondCom]|dataFriends["commonCom"]
    for i in ttl:
        rv[int(i)] = list(set(ttl[i]) & idsTotal)

    #преобразование в матрицу
    ks = list(rv.keys())
    R = {}
    for id in ks:
        R[str(id)] = [str(id)] + [int(ks[j] in rv[id]) for j in range(len(ks))]
    R = [["X"]+list(R.keys())]+list(R.values())

    return R ,lenTotal, lenFirst, lenCommon

In [ ]:
#запуск функции обработки и формирования матрицы смежности
R, lenTotal, lenFirst, lenCommon = processingData(firstCom, secondCom)

df = pd.DataFrame(R)
df.to_excel(r"drive/MyDrive/output/matrix2.xlsx", index=False)

#Словарь со списками, добавление параметров

In [ ]:
with open("drive/MyDrive/output/data_file.json", "r") as infile:
        data = infile.read()

dataFriends = json.loads(data)

In [ ]:
dataAll = {}
f = 0
for bigPub in dataFriends:
    dataAll[bigPub] = {}
    
    for item in dataFriends[bigPub]:
        # time.sleep(3)

        try:
            getUser_sub = vk.users.getSubscriptions(user_id = item, extend=0)
        except:
            continue

        dataAll[bigPub][item] = {}
        dataAll[bigPub][item]["publics"] = {}
        getUser_info = vk.users.get(user_id = item, fields="bdate, education, personal, city")
        getUser_friend = vk.friends.get(user_id = item)

        dataAll[bigPub][item]["info"] = getUser_info[0]
        dataAll[bigPub][item]["info"]["count_friends"] = getUser_friend["count"]

        grp_ids = ""

        for i in range(min(20,len(getUser_sub["groups"]["items"]))):   
          grp_ids += str(getUser_sub["groups"]["items"][i]) + ","

        grp_ids = grp_ids[:-1]

        try:
          getCom = vk.groups.getById(group_ids = grp_ids, fields='activity')
          for i in range(len(getCom)):
            dataAll[bigPub][item]["publics"][getUser_sub["groups"]["items"][i]] = getCom[i]["activity"]
        except:
          getCom = ""
          for i in range(len(getCom)):
              dataAll[bigPub][item]["publics"][getUser_sub["groups"]["items"][i]] = "Сайты"

        dataAll[bigPub][item]["info"]["count_publics"] = getUser_sub["groups"]["count"]

        dataAll[bigPub][item]["friends"] = dataFriends[bigPub][item]

In [ ]:
with open(r"drive/MyDrive/output/data_all.json", 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

##Определение типов агентов

In [ ]:
import datetime
from datetime import date

In [ ]:
with open("drive/MyDrive/output/data_all.json", "r") as infile:
        data = infile.read()

dataAll = json.loads(data)

In [ ]:
#квалиметрическая модель

def qualimetric_indicator(age, langs, city, edu, pub):
  
  qi= 0

  if age < 18: 
    qi += 0.05
  elif age >= 18 and age < 25:
    qi += 0.1
  elif age >= 25 and age < 35:
    qi += 0.15
  elif age >= 35 and age < 50:
    qi += 0.2
  else:
    qi += 0.25

  if langs < 2:
    qi +=0.05
  elif langs == 2:
    qi += 0.1
  elif langs == 3:
    qi += 0.15
  else:
    qi += 0.2

  if city == 4:
    qi +=0.4
  elif city == 3:
    qi +=0.3
  elif city == 2:
    qi +=0.2
  else:
    qi +=0.1

  if edu == 3:
    qi +=0.45
  elif edu == 2:
    qi +=0.3
  else:
    qi +=0.15

  if pub < 20:
    qi +=0.65
  elif pub >=20 and pub < 40:
    qi +=1.3
  elif pub >=40 and pub < 65:
    qi +=1.95
  else:
    qi +=2.6
  
  
  return qi/3.9

In [ ]:
def pre_qualimetric(user_info, getCityBig, getUni_list, getType):
 
  qi = 0
  today = date.today()

  if "bdate" in user_info["info"]:
    d = user_info["info"]["bdate"].split(".")
    if len(d) == 3:
      age = today.year - int(d[2]) - ((today.month, today.day) < (int(d[1]), int(d[0])))
    else:
      age = 18
  else:
    age = 18

  if "langs" in user_info["info"]:
    langs = len(user_info["info"]["langs"])
  else: 
    langs = 0

  if "city" in user_info["info"]:
    if user_info["info"]["city"]["id"] == 1 or user_info["info"]["city"]["id"] == 2:
      city = 4
    elif user_info["info"]["city"]["id"] in getCityBig["items"][0].values():
      city = 3
    else: 
      city = 2
  else:
    city = 1
    
  if "university" in user_info["info"]:
    if user_info["info"]["university"] in getUni_list:
      edu = 3
    else:
      edu = 2
  else:
    edu = 1

  pub = [0,0,0]
  cnt = [0,0,0]

  for i in user_info['publics']:

      try:
          cnt[int(getType[user_info['publics'][i]])]+=1
      except KeyError:
          cnt[1]+=1

  for i in range(3):
    try:
      pub[i] = cnt[i]/sum(cnt)*100 
    except ZeroDivisionError:
      pub[i] = 0

  qi = qualimetric_indicator(age, langs, city, edu, pub[2])

  if qi < 0.8:
    return "эмоциональный"
  else:
    return "аналитик"

In [ ]:
getCityBig = vk.database.getCities(country_id="1")

getUni = []
getUni1 = vk.database.getUniversities(city_id = "1")
getUni2 = vk.database.getUniversities(city_id = "2")

getUni.extend(getUni1["items"])
getUni.extend(getUni2["items"])

getUni_list = []

for i in range(len(getUni)):

  getUni_list.append(getUni[i]["id"])

getUni_list = set(getUni_list)

#Файл с таблицей типов пабликов
groups_types = pd.read_excel('drive/MyDrive/input/groups_types.xlsx')

getType = {}
for i in range(len(groups_types["Type"])):
    getType[groups_types["Type"][i]] = groups_types["Score"][i]

In [ ]:
for pubItem in dataAll:
  for item in dataAll[pubItem]:

    dataAll[pubItem][item]["param"] = {}

    try:
      dataAll[pubItem][item]["param"]["Th"] = 100/(0.6*dataAll[pubItem][item]["info"]["count_friends"] \
                                                 + 0.4*dataAll[pubItem][item]["info"]["count_publics"])
    except:
      dataAll[pubItem][item]["param"]["Th"] = 0
    
    dataAll[pubItem][item]["param"]["type"] = pre_qualimetric(dataAll[pubItem][item], getCityBig, getUni_list, getType)

In [ ]:
#Итоговый файл с типами агентов для загрузки в программу модели социальной сети

with open('drive/MyDrive/output/data_with_distribution.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

# Сбор статистики случайных групп
Сбор количества аналитиков и эмоциональных агентов в выбранных пабликах ВКонтакте

In [ ]:
group_name = "group_name" #название группы для сбора статистики
count = 200 #объем выборки

In [ ]:
getFirstCom = vk.groups.getById(group_id = group_name, fields='activity')
getFirstCom[0]["activity"]

In [ ]:
def collectingData(community, count):
  getFirstCom = vk.groups.getMembers(group_id = community, count = count, offset = '20000')

  dataFriends = {}
  countFirst = 0

  for i in range(len(getFirstCom["items"])):           
    try:
        getFriends = vk.friends.get(user_id = str(getFirstCom["items"][i]))
        g = {str(getFirstCom["items"][i]): getFriends['items']}
        dataFriends.update(g)
    except:
        countFirst += 1

  with open('drive/MyDrive/output/data_file.json', 'w') as outfile:
        json.dump(dataFriends, outfile)

collectingData(group_name, count)

In [ ]:
getCityBig = vk.database.getCities(country_id="1")

getUni = []
getUni1 = vk.database.getUniversities(city_id = "1")
getUni2 = vk.database.getUniversities(city_id = "2")

getUni.extend(getUni1["items"])
getUni.extend(getUni2["items"])

getUni_list = []

for i in range(len(getUni)):

  getUni_list.append(getUni[i]["id"])

getUni_list = set(getUni_list)

#Файл с таблицей типов пабликов
groups_types = pd.read_excel('drive/MyDrive/input/groups_types.xlsx')

getType = {}
for i in range(len(groups_types["Type"])):
    getType[groups_types["Type"][i]] = groups_types["Score"][i]

In [ ]:
with open("drive/MyDrive/output/data_file.json", "r") as infile:
        data = infile.read()

dataFriends = json.loads(data)

In [ ]:
#количество выгруженных пользователей
len(dataFriends)

128

In [ ]:
dataAll = {}
f = 0

dataAll = {}

for item in dataFriends:
  
    try:
        getUser_sub = vk.users.getSubscriptions(user_id = item, extend=0)
    except:
        continue

    dataAll[item] = {}
    dataAll[item]["publics"] = {}

    getUser_info = vk.users.get(user_id = item, fields="bdate, education, personal, city")
    getUser_friend = vk.friends.get(user_id = item)

    dataAll[item]["info"] = getUser_info[0]
    dataAll[item]["info"]["count_friends"] = getUser_friend["count"]

    grp_ids = ""

    for i in range(min(20,len(getUser_sub["groups"]["items"]))):   
      grp_ids += str(getUser_sub["groups"]["items"][i]) + ","

    grp_ids = grp_ids[:-1]

    try:
      getCom = vk.groups.getById(group_ids = grp_ids, fields='activity')
      for i in range(len(getCom)):
        dataAll[item]["publics"][getUser_sub["groups"]["items"][i]] = getCom[i]["activity"]
    except:
     for i in range(len(getCom)):
        dataAll[item]["publics"][getUser_sub["groups"]["items"][i]] = "Сайты"

    dataAll[item]["info"]["count_publics"] = getUser_sub["groups"]["count"]

    dataAll[item]["friends"] = dataFriends[item]

with open('drive/MyDrive/output/data_all.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

In [ ]:
with open("drive/MyDrive/output/data_all.json", "r") as infile:
        data = infile.read()

dataAll = json.loads(data)

In [ ]:
for pubItem in dataAll:
  for item in dataAll:

    dataAll[item]["param"] = {}

    dataAll[item]["param"]["Th"] = 100/(0.6*dataAll[item]["info"]["count_friends"] \
                                                 + 0.4*dataAll[item]["info"]["count_publics"])
    dataAll[item]["param"]["type"] = pre_qualimetric(dataAll[item], getCityBig, getUni_list, getType)

In [ ]:
with open('drive/MyDrive/output/data_all.json', 'w') as outfile:
    json.dump(dataAll, outfile, ensure_ascii=False)

In [ ]:
with open("drive/MyDrive/output/data_all.json", "r") as infile:
        data_json = infile.read()
data_json= json.loads(data_json)
type(data_json)

Th = []
mem = []
disc = []

for item in data_json:
  Th.append(data_json[item]["param"]["Th"])
  if data_json[item]["param"]["type"] == "аналитик":
    mem.append(3)
    disc.append(0.7)
  else:
    mem.append(1)
    disc.append(1)


#Итоговое количество аналитиков (3) и эмоциональных агентов (1) среди выбранных
#подписчиков исследуемого паблика
c = Counter(mem)
c

Counter({1: 126, 3: 2})